In [3]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import time
import sys

In [4]:
def next_page(driver):
    try:
        driver.find_element_by_xpath('.//div[@class="_3djM0GaD"]').click()
        return False
    # except ElementClickInterceptedException as e1:
    #     driver.find_element_by_xpath('.//button[@class="_3VKU_-kL"]').click()
    #     driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()
    except NoSuchElementException as e2: # last page of all reviews
        return True

In [5]:
# def get_url_at_page(url, page):
#     if page > 0:
#         pos = url.find("Reviews-")
#         result = url[:pos + 7] + "-or" + str(page * 5) + url[pos + 7:]
#     else:
#         result = url
#     return result

In [9]:
attractions = pd.read_csv('attractions.csv', index_col=0)
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)

for i, attraction in attractions.iterrows():
    url = attraction['name']
    # print(url)
    if attraction['done'] > 0:
        continue
    driver.get(url)
    attraction_name = driver.find_element_by_xpath("//h1[@data-automation='mainH1']").text

    print(attraction_name)
    isRun = True
    while isRun:
        isRun = False
        columns = ['page', 'user', 'country', 'date_post', 'date_stay', 'rating', 'title', 'review_text']
        try:
            reviews = pd.read_csv("reviews_" + attraction_name + ".csv", index_col=False)
            if len(reviews) == 0:
                page = 1
            else:
                page = int(reviews.tail(1)['page']) + 1
        except FileNotFoundError as e:
            reviews = pd.DataFrame(columns=columns)
            reviews.to_csv("reviews_" + attraction_name + ".csv", index=False)
            page = 0

        # driver.get(get_url_at_page(url, page))
        # change the value inside the range to save more or less reviews
        isStop = False
        while not isStop:
            # expand the review
            time.sleep(0.6)
            try:
                # isExpand = True
                # while isExpand:
                #     try:
                #         driver.find_element_by_xpath(".//span[@class='_1l3JzGX1']").click()
                #         isExpand = False
                #     except ElementNotInteractableException as e1:
                #         time.sleep(0.3)
                #     except NoSuchElementException as e2:
                #         next_page(driver)
                #         isExpand = False
                container = driver.find_element_by_xpath("//div[@class='_1c8_1ITO']")
                reviews_list = container.find_elements_by_xpath("./*")

                for j in range(len(reviews_list[:-1])):
                    # sort data out
                    details = reviews_list[j].text.split("\n")
                    print(details)
                    user = details[0]
                    country = ''.join([i for i in details[1].split("contributions",1)[0] if not i.isdigit()])
                    date_post = details[details.index("This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC.")-1].split(" ",1)[1]
                    rating = reviews_list[j].find_elements_by_tag_name("svg")[2].get_attribute('title').split(" ")[0]
                    title = details[3]
                    date_stay = details[4]
                    review_text = "".join([i for i in details[5:details.index("This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC.")-1]])
                    review_text = review_text.split("Read more",1)[0]
                    print(review_text)
                    new_entry = pd.DataFrame(
                        [[page, user, country, date_post, date_stay, rating, title, review_text]],
                        columns=columns)
                    # print(new_entry)
                    # if not (date_post[-4:-2] == "20" and int(date_post[-4:]) < 2018): # date stop condition
                    #     new_entry.to_csv("reviews_" + attraction_name + ".csv", mode='a', header=False, index=False)
                    new_entry.to_csv("reviews_" + attraction_name + ".csv", mode='a', header=False, index=False)
                    # if date_post[-4:-2] == "20" and int(date_post[-4:]) < 2018:
                    #     isStop = True
                    #     break
                if next_page(driver):
                    break
                page += 1
            except StaleElementReferenceException as e:
                print('stale')
                isRun = True
                break
    attractions.loc[i, 'done'] = len(pd.read_csv("reviews_" + attraction_name + ".csv"))
    attractions.to_csv('attractions.csv')


https://www.tripadvisor.com.sg/Attraction_Review-g294264-d2439664-Reviews-Universal_Studios_Singapore-Sentosa_Island.html
Universal Studios Singapore


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1c8_1ITO']"}
  (Session info: chrome=98.0.4758.102)
